##### Imports:

In [2]:
from utils import separate_for_training, preprocess, calculate_emission_from_chroma, calculate_mu_from_chroma, calculate_transition_probabilites
import pickle
from chroma import get_chromagram
import pandas as pd
from tqdm import tqdm
import numpy as np

##### Steps:

1. Training / Testing Data Split
2. Create Chromagram from Training Data
3. Create HMM Initialization Components
    - Initial State Probabilities
    - Transition Probability Matrix
    - Mu Value
    - Emission Matrix
4. Create HMM Object
5. Fit / Train HMM

##### Training / Test Data Split:

In [3]:
# Load data and split into training and test
piece_name_dict = preprocess('dataset.pkl')

training_piece_names, _, test_piece_names = separate_for_training(piece_name_dict, 0.8, 0.)

with open(r"dataset.pkl", 'rb') as data:
    midi_data = pickle.load(data)

100%|██████████| 5762/5762 [00:10<00:00, 530.71it/s] 


##### Create Chromagram from Training Data:

In [4]:
song_chromagrams = []
for song_name in tqdm(list(training_piece_names)):
    indiv_chroma = get_chromagram(song_name, midi_data)
    song_chromagrams.append(indiv_chroma)

chromagram = pd.concat(song_chromagrams)
chromagram.head(6)

  0%|          | 0/4609 [00:00<?, ?it/s]

100%|██████████| 4609/4609 [01:15<00:00, 61.28it/s] 


,C,C#,D,D#,E,F,F#,G,G#,A,A#,B,Chord Actual
0,0,0,0,64,0,0,154,0,67,0,0,85,Bm
1,0,0,0,64,0,0,154,0,67,0,0,85,Bm
2,0,0,0,64,0,0,154,0,67,0,0,85,Bm
3,0,0,0,64,0,0,154,0,67,0,0,85,Bm
4,0,0,0,64,0,0,154,0,67,0,0,85,Bm
5,0,0,0,64,0,0,154,0,67,0,0,85,Bm


##### Create HMM Components:

###### Initial State Probabilities:

In [5]:
def calculate_init_prob(song_chromagrams):
    first_chords = []
    for song_chroma in tqdm(song_chromagrams):
        first_chord = song_chroma['Chord Actual'].values[0]
        first_chords.append(first_chord)
    
    first_chord_counts = np.unique(first_chords, return_counts=True)

    return pd.Series(first_chord_counts[1]/first_chord_counts[1].sum(), index=first_chord_counts[0])

In [6]:
initial_state_probabilties = calculate_init_prob(song_chromagrams)
initial_state_probabilties

100%|██████████| 4609/4609 [00:00<00:00, 15064.44it/s]


A        0.020395
A#       0.000217
A#dim    0.000217
B        0.123454
Bm       0.153829
C        0.000217
C#dim    0.000217
C#m      0.060317
D        0.117379
D#m      0.020178
E        0.119549
Em       0.064656
F#       0.021046
F#m      0.021480
G        0.115426
G#m      0.161423
dtype: float64

###### Transition Matrix:

In [7]:
transition_prob_matrix = calculate_transition_probabilites(chromagram)
transition_prob_matrix

following_chords,A,A#,A#dim,B,Bm,C,C#dim,C#m,D,D#,...,Em,F,F#,F#m,G,G#,G#m,dim,<E>,<S>
initial_chords,,,,,,,,,,,,,,,,,,,,,
A,0.862012,0.000000,0.000072,0.005137,0.043271,0.000072,0.002894,0.002677,0.018524,0.000000,...,0.010781,0.000000,0.001013,0.010709,0.021201,0.000217,0.009479,0.000072,0.0,0
A#,0.000000,0.875000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.083333,...,0.041667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0
A#dim,0.000000,0.000000,0.880829,0.012953,0.015544,0.000000,0.000000,0.007772,0.007772,0.000000,...,0.012953,0.000000,0.005181,0.000000,0.012953,0.000000,0.023316,0.000000,0.0,0
B,0.000448,0.000000,0.000000,0.862118,0.004387,0.000000,0.000000,0.014683,0.009759,0.000000,...,0.002149,0.000000,0.048706,0.000627,0.003223,0.000000,0.028740,0.000000,0.0,0
Bm,0.043795,0.000000,0.000000,0.002270,0.863542,0.000067,0.000000,0.000868,0.023566,0.000000,...,0.011750,0.000000,0.000067,0.021831,0.026771,0.000000,0.002871,0.000000,0.0,0
C,0.006383,0.000000,0.000000,0.008511,0.004255,0.861702,0.002128,0.000000,0.004255,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.110638,0.000000,0.000000,0.000000,0.0,0
C#dim,0.005587,0.000000,0.000000,0.016760,0.027933,0.000000,0.879888,0.005587,0.011173,0.000000,...,0.000000,0.000000,0.002793,0.002793,0.019553,0.000000,0.011173,0.000000,0.0,0
C#m,0.000444,0.000000,0.000000,0.010206,0.002810,0.000000,0.000000,0.860671,0.003846,0.000000,...,0.001775,0.000000,0.013312,0.000148,0.002958,0.000000,0.047330,0.000148,0.0,0
D,0.052599,0.000000,0.000000,0.003501,0.027466,0.000359,0.000000,0.001526,0.863118,0.000000,...,0.015079,0.005116,0.000449,0.004308,0.017682,0.000000,0.003949,0.000000,0.0,0


###### Mu Value:

In [8]:
mu = calculate_mu_from_chroma(chromagram)
mu

C     11.489865
C#    36.953119
D     25.102938
D#    34.313296
E     29.482048
F     16.377439
F#    44.676278
G     16.828030
G#    40.337249
A     21.984540
A#    24.701546
B     49.716330
dtype: float64

###### Emission Matrix:

In [9]:
emission = calculate_emission_from_chroma(chromagram)